In [2]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import copy
import random
import time
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from tqdm import tqdm
from torchvision.models import ResNet
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, utils, datasets
from argparse import ArgumentParser
from torchvision import transforms as tt
import csv
from collections import defaultdict
import torch.nn.utils.prune as prune

# set manual seed for reproducibility
# seed = 42

# # general reproducibility
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)

# gpu training specific
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from PIL import Image
import numpy as np
import torchvision.transforms.functional as T
import glob, random

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, csv_file, idxs, transform=None):
        self.dataset = pd.read_csv(csv_file, header=None)
        self.transform = transform

        self.idxs = list(idxs)


    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        # Extract 11 input values and 1 target value
        inputs = self.dataset.iloc[self.idxs[item], :-1].values.astype(np.float32)
        target = self.dataset.iloc[self.idxs[item], -1].astype(np.float32)

        return inputs, target

In [4]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, learning_rate, epochs, idxs, sch_flag):
        self.train_loader = DataLoader(CustomDataset(dataset, idxs), batch_size=batchSize, shuffle=True)
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.sch_flag = sch_flag

    def train(self, model):

        criterion = nn.MSELoss()
        # optimizer = torch.optim.SGD(model.parameters(), lr=self.learning_rate, momentum=0.95, weight_decay = 5e-4)
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate)
        # if self.sch_flag == True:
        #    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5)
        # my_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
        e_loss = []
        for epoch in range(1, self.epochs + 1):

            train_loss = 0.0

            model.train()
            for data, labels in self.train_loader:
                if data.size()[0] < 2:
                    continue;

                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                # clear the gradients
                optimizer.zero_grad()
                # make a forward pass
                output = model(data)
                # calculate the loss
                loss = criterion(output, labels.unsqueeze(1))
                # do a backwards pass
                loss.backward()
                # perform a single optimization step
                optimizer.step()
                # update training loss
                train_loss += loss.item() * data.size(0)
                # if self.sch_flag == True:
                #  scheduler.step(train_loss)
            # average losses
            train_loss = train_loss / len(self.train_loader.dataset)
            e_loss.append(train_loss)

            # self.learning_rate = optimizer.param_groups[0]['lr']

        total_loss = sum(e_loss) / len(e_loss)

        return model.state_dict(), total_loss

In [5]:
"""## Testing Loop"""


def testing(model, dataset, bs, criterion, num_classes, classes):
    # test loss

    test_loss = 0.0
    idxs = np.zeros(1614)
    for i in range(6):
        idxs[i * 269: (i + 1) * 269] = np.arange((i + 1) * 2688 - 269, (i + 1) * 2688)

    idxs = np.arange(14515, 16128)
        
    test_loader = DataLoader(CustomDataset(dataset, idxs.astype(int)), batch_size=bs, shuffle=True)
    l = len(test_loader)
    model.eval()
    mae = 0
    for data, labels in test_loader:

        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
            
        output = model(data)
        loss = criterion(output, labels.unsqueeze(1))
        test_loss += loss.item() * data.size(0)

        # print(output)
        # print(labels)
        # print(sum(output - labels))

        mae += sum(abs(output.squeeze() - labels.squeeze())).item()

    # avg test loss
    test_loss = test_loss / len(test_loader.dataset)

    return mae / len(test_loader.dataset), test_loss

In [6]:
"""### Server Side Training

Following Algorithm 1 from the paper
"""

def training(model, rounds, batch_size, lr, ds, data_dict, C, K, E, plt_title, plt_color, cifar_data_test,
             test_batch_size, criterion, num_classes, classes_test, sch_flag, filename, cr):
    """
    Function implements the Federated Averaging Algorithm from the FedAvg paper.
    Specifically, this function is used for the server side training and weight update

    Params:
      - model:           PyTorch model to train
      - rounds:          Number of communication rounds for the client update
      - batch_size:      Batch size for client update training
      - lr:              Learning rate used for client update training
      - ds:              Dataset used for training
      - data_dict:       Type of data partition used for training (IID or non-IID)
      - C:               Fraction of clients randomly chosen to perform computation on each round
      - K:               Total number of clients
      - E:               Number of training passes each client makes over its local dataset per round
      - tb_writer_name:  Directory name to save the tensorboard logs
    Returns:
      - model:           Trained model on the server
    """
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=1-cr)
        elif isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=1-cr)
        elif isinstance(module, torch.nn.BatchNorm2d):
            prune.l1_unstructured(module, name='weight', amount=1-cr)
    
    
    # global model weights
    global_weights = model.state_dict()
    

    # training loss
    train_loss = []
    test_loss = []
    test_accuracy = []
    best_accuracy = 9999
    # measure time
    start = time.time()

    for curr_round in range(1, rounds + 1):

        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                prune.remove(module, name='weight')
            elif isinstance(module, torch.nn.Linear):
                prune.remove(module, name='weight',)
            elif isinstance(module, torch.nn.BatchNorm2d):
                prune.remove(module, name='weight')

        w, local_loss = [], []
        # Retrieve the number of clients participating in the current training
        m = max(int(C * K), 1)
        # Sample a subset of K clients according with the value defined before
        S_t = np.random.choice(range(K), m, replace=False)
        # For the selected clients start a local training
        for k in tqdm(S_t):
            # Compute a local update
            local_update = ClientUpdate(dataset=ds, batchSize=batch_size, learning_rate=lr, epochs=E, idxs=data_dict[k],
                                        sch_flag=sch_flag)
            # Update means retrieve the values of the network weights
            weights, loss = local_update.train(model=copy.deepcopy(model))

            w.append(copy.deepcopy(weights))
            local_loss.append(copy.deepcopy(loss))
        # lr = 0.999*lr
        # updating the global weights
        weights_avg = copy.deepcopy(w[0])
        for k in weights_avg.keys():
            for i in range(1, len(w)):
                weights_avg[k] += w[i][k]

            weights_avg[k] = torch.div(weights_avg[k], len(w))

        global_weights = weights_avg

        if curr_round == 100:
            lr = lr / 2
            E = E - 1

        if curr_round == 150:
            lr = lr / 2
            E = E - 2

        if curr_round == 175:
            lr = lr / 5
            E = E - 3

        # move the updated weights to our model state dict
        model.load_state_dict(global_weights)

        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                prune.l1_unstructured(module, name='weight', amount=1-cr)
            elif isinstance(module, torch.nn.Linear):
                prune.l1_unstructured(module, name='weight', amount=1-cr)
            elif isinstance(module, torch.nn.BatchNorm2d):
                prune.l1_unstructured(module, name='weight', amount=1-cr)

        # loss
        loss_avg = sum(local_loss) / len(local_loss)
        # print('Round: {}... \tAverage Loss: {}'.format(curr_round, round(loss_avg, 3)), lr)
        train_loss.append(loss_avg)

        t_accuracy, t_loss = testing(model, cifar_data_test, test_batch_size, criterion, num_classes, classes_test)
        test_accuracy.append(t_accuracy)
        test_loss.append(t_loss)

        if best_accuracy > t_accuracy:
            best_accuracy = t_accuracy
        # torch.save(model.state_dict(), plt_title)
        if filename is not None:
            with open(filename, 'a') as f:
                # create the csv writer
                writer = csv.writer(f)

                # write a row to the csv file
                writer.writerow([curr_round, loss_avg, t_loss, t_accuracy, best_accuracy])
        print(f"Current Round: {curr_round}, Average Loss: {loss_avg}, Test Loss: {t_loss}, Test Accuracy: {t_accuracy}, Best: {best_accuracy}")
        # print('best_accuracy:', best_accuracy, '---Round:', curr_round, '---lr', lr, '----localEpocs--', E)

    end = time.time()
    plt.rcParams.update({'font.size': 8})
    # fig, ax = plt.subplots()
    # x_axis = np.arange(1, rounds + 1)
    # y_axis1 = np.array(train_loss)
    # y_axis2 = np.array(test_accuracy)
    # y_axis3 = np.array(test_loss)

    # ax.plot(x_axis, y_axis1, 'tab:' + 'green', label='train_loss')
    # ax.plot(x_axis, y_axis2, 'tab:' + 'blue', label='test_accuracy')
    # ax.plot(x_axis, y_axis3, 'tab:' + 'red', label='test_loss')
    # ax.legend(loc='upper left')
    # ax.set(xlabel='Number of Rounds', ylabel='Train Loss',
    #        title=plt_title)
    # ax.grid()
    # fig.savefig(plt_title+'.jpg', format='jpg')
    print("Training Done!")
    print(f"Best Accuracy: {best_accuracy}")
    print("Total time taken to Train: {}".format(end - start))

    return model

In [7]:
data_dict = {}
for i in range(6):
    data_dict[i] = np.arange(i * 2688, (i + 1) * 2688 - 269)

In [8]:
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

In [52]:
# H = [args.commrounds, args.clientfr, args.numclient, args.clientepochs, args.clientbs, args.clientlr]
for i in range(5):
    model = RegressionModel(11)
    model.cuda()
    criterion = nn.MSELoss()
    cr = 0.25
    num = 1
    filename = f"BuildingFL_CR{cr}_{num}"
    while os.path.isfile(filename):
        print('Name is taken...trying again...')
        num += 1
        filename = f"BuildingFL_CR{cr}_{num}"


    training(model, 200, 16, 0.001, './data/dataloader.csv', data_dict, 1.0, 6, 1, 'plot_str', "green", './data/dataloader.csv', 2, criterion, 10, 10, False, filename, cr)

100%|██████████| 6/6 [00:01<00:00,  3.48it/s]


Current Round: 1, Average Loss: 716.0542008201659, Test Loss: 1885.3520388668207, Test Accuracy: 39.854440505915264, Best: 39.854440505915264


100%|██████████| 6/6 [00:01<00:00,  3.57it/s]


Current Round: 2, Average Loss: 381.92865688268256, Test Loss: 1210.8416231999167, Test Accuracy: 30.888493704529942, Best: 30.888493704529942


100%|██████████| 6/6 [00:01<00:00,  3.48it/s]


...

Current Round: 198, Average Loss: 49.71550360030833, Test Loss: 85.87844011377742, Test Accuracy: 6.488156270212696, Best: 6.407514775404995


100%|██████████| 6/6 [00:01<00:00,  3.48it/s]


Current Round: 199, Average Loss: 50.04502063348173, Test Loss: 84.64384446150012, Test Accuracy: 6.414830450083982, Best: 6.407514775404995


100%|██████████| 6/6 [00:01<00:00,  3.68it/s]


Current Round: 200, Average Loss: 50.88039788855523, Test Loss: 85.75224516216707, Test Accuracy: 6.4813884749288455, Best: 6.407514775404995
Training Done!
Best Accuracy: 6.407514775404995
Total time taken to Train: 426.51221799850464


In [51]:
for i in range(5):
    model = RegressionModel(11)
    model.cuda()
    criterion = nn.MSELoss()
    cr = 0.1
    num = 1
    filename = f"BuildingFL_CR{cr}_{num}"
    while os.path.isfile(filename):
        print('Name is taken...trying again...')
        num += 1
        filename = f"BuildingFL_CR{cr}_{num}"


    training(model, 200, 16, 0.001, './data/dataloader.csv', data_dict, 1.0, 6, 1, 'plot_str', "green", './data/dataloader.csv', 2, criterion, 10, 10, False, filename, cr)

100%|██████████| 6/6 [00:01<00:00,  3.51it/s]


Current Round: 1, Average Loss: 1506.1762074405733, Test Loss: 5751.266207972719, Test Accuracy: 74.5306768405541, Best: 74.5306768405541


100%|██████████| 6/6 [00:01<00:00,  3.44it/s]


Current Round: 2, Average Loss: 1272.4035921657942, Test Loss: 5723.42590044629, Test Accuracy: 74.33152765796382, Best: 74.33152765796382


100%|██████████| 6/6 [00:01<00:00,  3.44it/s]


...

Current Round: 198, Average Loss: 86.08817902277599, Test Loss: 346.60022177752177, Test Accuracy: 10.605109837566314, Best: 9.557880408701843


100%|██████████| 6/6 [00:01<00:00,  3.71it/s]


Current Round: 199, Average Loss: 89.54465092504847, Test Loss: 341.37585736308915, Test Accuracy: 10.483303455319694, Best: 9.557880408701843


100%|██████████| 6/6 [00:01<00:00,  3.74it/s]


Current Round: 200, Average Loss: 91.51853850348783, Test Loss: 341.5566074869225, Test Accuracy: 10.479179538315556, Best: 9.557880408701843
Training Done!
Best Accuracy: 9.557880408701843
Total time taken to Train: 417.09344029426575


In [13]:
one_dict = {}
one_dict[0] = np.arange(14515)
num=2
for i in range(4):
    criterion = nn.MSELoss()
    model = RegressionModel(11)
    model.cuda()
    filename = f"BuildingFL_NOFL{num}"
    while os.path.isfile(filename):
        print('Name is taken...trying again...')
        num += 1
        filename = f"BuildingFL_NOFL{num}"
    training(model, 100, 16, 0.001, './data/dataloader.csv', one_dict, 1.0, 1, 1, 'plot_str', "green", './data/dataloader.csv', 16, criterion, 10, 10, False, filename, 1.0)

100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


Current Round: 1, Average Loss: 194.44743538205066, Test Loss: 62.22600950842974, Test Accuracy: 6.395590212798902, Best: 6.395590212798902


100%|██████████| 1/1 [00:01<00:00,  1.69s/it]


Current Round: 2, Average Loss: 90.09667549205574, Test Loss: 49.83233490274031, Test Accuracy: 5.552458146456525, Best: 5.552458146456525


100%|██████████| 1/1 [00:01<00:00,  1.68s/it]


...

Current Round: 98, Average Loss: 45.365660815316154, Test Loss: 35.63328605457928, Test Accuracy: 4.829298233439001, Best: 4.492055380928243


100%|██████████| 1/1 [00:01<00:00,  1.55s/it]


Current Round: 99, Average Loss: 45.213722521200125, Test Loss: 36.54904365421302, Test Accuracy: 4.837247315209916, Best: 4.492055380928243


100%|██████████| 1/1 [00:01<00:00,  1.58s/it]

Current Round: 100, Average Loss: 45.093515198878734, Test Loss: 33.820873524192244, Test Accuracy: 4.5270242803376135, Best: 4.492055380928243
Training Done!
Best Accuracy: 4.492055380928243
Total time taken to Train: 181.89389157295227


In [54]:
16128 * .1

1612.8000000000002